這個筆記本提供載入和儲存外部來源資料的方法。

# 本機檔案系統

## 從本機檔案系統上傳檔案

<code>files.upload</code> 會傳回已上傳檔案的字典。
字典會使用檔案名稱做為索引鍵，值則是上傳的資料。

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## 將檔案下載至本機檔案系統

<code>files.download</code> 會透過瀏覽器下載檔案至本機電腦。


In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Google 雲端硬碟

你可透過多種方式存取雲端硬碟中的檔案，包括：
- 在執行階段的虛擬機器中掛接 Google 雲端硬碟
- 在 <a href="https://docs.iterative.ai/PyDrive2/">PyDrive2</a> 等 API 前後使用包裝函式
- 使用<a href="https://developers.google.com/drive/v3/web/about-sdk">原生 REST API</a>



個別範例位於下方。

## 在本機環境掛接 Google 雲端硬碟

下列範例會顯示如何在執行階段使用授權碼掛接 Google 雲端硬碟，並在硬碟上寫入和讀取檔案。執行之後，你會在 <a href="https://drive.google.com/">https://drive.google.com/</a> 看到新檔案 &#40;<code>foo.txt</code>&#41;。

此功能僅支援讀取、寫入及移動檔案；如要透過程式修改共用設定或其他中繼資料，請使用下列任一選項。

<strong>注意：</strong>在檔案瀏覽器中使用「掛接雲端硬碟」按鈕時，僅由目前使用者編輯的筆記本不需要驗證碼。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## PyDrive2

下列範例說明了如何使用 PyDrive2 進行驗證及上傳/下載檔案。更多範例請參閱 <a href="https://docs.iterative.ai/PyDrive2/">PyDrive2 說明文件</a>。

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

驗證並建立 PyDrive2 用戶端。


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

建立並上傳文字檔案。


In [ ]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 14vDAdqp7BSCQnoougmgylBexIr2AQx2T


依 ID 載入檔案，並列印其內容。


In [ ]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## Drive Rest API

如要使用 Drive API，我們必須先驗證並建立 API 用戶端。


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

透過這個用戶端，我們即可使用 <a href="https://developers.google.com/drive/v3/reference/">Google 雲端硬碟 API 參考資料</a>中的任何函式。範例如下：


### 使用 Python 的資料建立新的雲端硬碟檔案

首先，建立要上傳的本機檔案。

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

使用 <a href="https://developers.google.com/drive/v3/reference/files/create"><code>files.create</code></a> 方法上傳檔案。如要進一步瞭解上傳檔案的方法，請參閱<a href="https://developers.google.com/drive/v3/web/manage-uploads">開發人員說明文件</a>。

In [ ]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1Cw9CqiyU6zbXFD9ViPZu_3yX-sYF4W17


執行上方的儲存格後，你會在 <a href="https://drive.google.com/">https://drive.google.com/</a> 上看到名為「Sample file」的新檔案。

### 將雲端硬碟檔案的資料下載至 Python

下載我們剛才上傳的檔案。

In [ ]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


如要下載其他檔案，請將上述的 <code>file&#95;id</code> 設為該檔案的 ID，例如「1uBtlaggVyWshwcyP6kEI-y&#95;W3P8D26sz」。

# Google 試算表


## Google 試算表 Workspace 擴充功能

Workspace 擴充功能「<a href="https://workspace.google.com/u/0/marketplace/app/sheets_to_colab/945625412720">Sheets to Colab</a>」可讓你透過試算表使用者介面，直接將 Google 試算表資料匯入 Colab。點選連結即可前往 Workspace 擴充功能「Sheets to Colab」頁面瞭解詳情。

## 使用 gspread 與 Google 試算表互動

你也可以使用開放原始碼 <a href="https://github.com/burnash/gspread"><code>gspread</code></a> 程式庫與 Google 試算表互動。以下程式碼說明如何設定及驗證 <code>gspread</code>。

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

下列是一小部分的 <code>gspread</code> 範例。你可以在 <a href="https://github.com/burnash/gspread#more-examples"><code>gspread</code> GitHub 頁面</a>中找到其他範例。

### 使用 Python 的資料建立新的試算表

In [ ]:
sh = gc.create('My cool spreadsheet')

執行上方的儲存格後，你會在 <a href="https://sheets.google.com/">https://sheets.google.com</a> 看到名為「My cool spreadsheet」的新試算表。

開啟新工作表，並新增一些隨機性資料。

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1dsQeN0YzXuM387l_CuyEbsYzL2ew9TJFzR-E-RQnwxs',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2}

### 將試算表的資料下載至 Python 做為 Pandas DataFrame

讀回我們在上方插入的隨機性資料，並將結果轉換為 <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html">Pandas DataFrame</a>。

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)

[['6', '3', '4'], ['7', '2', '1']]


,0,1,2
0,6,3,4
1,7,2,1


# InteractiveSheet

你現在可以使用 <code>InteractiveSheet</code> 程式庫，在 Colab 中嵌入即時 Google 試算表。也就是說，你可以在 Google 試算表中建立及編輯資料，並透過 Colab 提供的 Pandas DataFrame 將資料順暢地整合至筆記本中。

In [ ]:
from google.colab import sheets

# Create a new interactive sheet and add data to it.
sheet = sheets.InteractiveSheet()

In [ ]:
# Get a Pandas DataFrame from the selected worksheet
df = sheet.as_df()

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))

# Create a new sheet and include the column names as the first row.
sheet = sheets.InteractiveSheet(df=df, title='foo', include_column_headers=True)

In [ ]:
# Push data from Colab to the selected worksheet
df2 = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))
sheet.update(df=df)

In [ ]:
# Display the sheet in the output of the current cell
sheet.display()

# Google Cloud Storage &#40;GCS&#41;

如要將 Colaboratory 搭配 GCS 使用，必須建立 <a href="https://cloud.google.com/storage/docs/projects">Google Cloud 專案</a>，或是使用現有的專案。

在下方指定專案 ID：

In [ ]:
project_id = 'Your_project_ID_here'

<a href="https://cloud.google.com/storage/docs/buckets">值區</a>中已包含 GCS 中的檔案。

值區的名稱必須獨一無二，因此我們在此建立一個新名稱。

In [ ]:
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

我們必須進行驗證才能存取 GCS。

In [ ]:
from google.colab import auth
auth.authenticate_user()

GCS 可透過 <code>gsutil</code> 指令列公用程式或是原生 Python API 存取。

## `gsutil`

首先，設定 <code>gsutil</code>，以透過 <code>gcloud</code> 使用上方所指定的專案。

In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


建立要上傳的本機檔案。

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

建立存放上傳檔案 &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/mb">說明文件</a>&#41; 的值區。

In [ ]:
!gsutil mb gs://{bucket_name}

Creating gs://colab-sample-bucket-44971372-baaf-11e7-ae30-0242ac110002/...


將檔案複製到新值區 &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cp">說明文件</a>&#41;。

In [ ]:
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Copying file:///tmp/to_upload.txt [Content-Type=text/plain]...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


dump 最近複製檔案的內容，確保一切運作正常 &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cat">說明文件</a>&#41;。


In [ ]:
!gsutil cat gs://{bucket_name}/to_upload.txt

my sample file

In [ ]:
# @markdown 上傳完成後，資料會顯示在專案的 Google Cloud Storage 瀏覽器中：
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


最後，我們要下載剛才在上方範例中所上傳的檔案。這跟在 <code>gsutil cp</code> 指令中反轉順序一樣簡單。

In [ ]:
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt

# 列印結果以確認轉移成功。
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket483f20dc-baaf-11e7-ae30-0242ac110002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

## Python API

這些程式碼片段是以<a href="https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py">更大的範例</a>為根據，範例中展示了 API 的其他用途。

首先，請建立服務用戶端。

In [ ]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

建立要上傳的本機檔案。

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

在上方指定的專案中建立值區。

In [ ]:
# 使用不同於上方 gsutil 範例且獨一無二的值區名稱。
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

Done


將檔案上傳到新建立的值區。

In [ ]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name,
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Upload complete


In [ ]:
# @markdown 上傳完成後，資料會顯示在專案的 Google Cloud Storage 瀏覽器中：
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


下載剛才上傳的檔案。

In [ ]:
from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


檢查已下載的檔案。


In [ ]:
!cat /tmp/downloaded_from_gcs.txt

my sample file